<a href="https://colab.research.google.com/github/K18UHFT65RK9GitLo/ATCabbDiss/blob/main/%D0%9E%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0_Doc_%3ECh_%3EEm_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#@title 1. Installations and connections

In [ ]:
!pip install -q openai==1.58.1 langchain==0.3.13 langchain-openai==0.2.14 langchain_community==0.3.13 faiss-cpu==1.9.0.post1

In [ ]:
!pip install -q python-docx

In [ ]:
!pip install -q PyPDF2

In [ ]:
!pip install -q -U langchain-openai

#@title 1.1. Подключение к OpenAI

In [ ]:
# 1.1. Подключение к OpenAI

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.document_loaders import TextLoader
import re
import requests
import textwrap
import openai
import os
from google.colab import userdata
from openai import OpenAI
import os
import json
import io
import docx
import PyPDF2
import xml.etree.ElementTree as ET



# Использование ключа API от ProxyAPI
key = userdata.get('myproxy')
os.environ["OPENAI_API_KEY"] = key

# Адрес сервера ProxyAPI
base_url = 'https://api.proxyapi.ru/openai/v1'
os.environ["OPENAI_BASE_URL"] = base_url
client = OpenAI()       # создается экземпляр класса OpenAI

In [ ]:
# 1.2 Подключение к диску
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1.3  Настройка пути к файлу и базового пути для сохранения

FILE_PATH --->  путь к папке входящего документа фориат:  DOCX,  PDF,  XML,  TXT

OUTPUT_BASE_PATH  --->  путь к папке с чанками документа формат .json


In [ ]:
# 1.3  Настройка пути к файлу и базового пути для сохранения

#FILE_PATH -> путь к папке входящего документа фориат: DOCX, PDF, XML, TXT
FILE_PATH = "/content/drive/MyDrive/СТАЖИРОВКА_ЖКХ/ВХОДНЫЕ_УО_АСПЕКТ/d_asp/ФЗ от 27.07.2010 № 190-ФЗ-О теплоснабжении.docx"  #@param {type:"string"}

#OUTPUT_BASE_PATH -> путь к папке с чанками документа формат .json
OUTPUT_BASE_PATH = "/content/drive/MyDrive/СТАЖИРОВКА_ЖКХ/Отчёты"  #@param {type:"string"}

#@title 2. Text to chunk

In [ ]:
# 2. Функции для загрузки и конвертации документов в текст

def load_text_from_doc(file_path):
    """Загружает текст из DOCX файла."""
    try:
        doc = docx.Document(file_path)
        full_text = []
        for paragraph in doc.paragraphs:
            full_text.append(paragraph.text)
        return '\n'.join(full_text)
    except Exception as e:
        print(f"Ошибка при чтении DOCX файла: {e}")
        return None

def load_text_from_pdf(file_path):
    """Загружает текст из PDF файла."""
    try:
        with open(file_path, 'rb') as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            full_text = []
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                full_text.append(page.extract_text())
            return '\n'.join(full_text)
    except Exception as e:
        print(f"Ошибка при чтении PDF файла: {e}")
        return None

def load_text_from_xml(file_path):
    """Загружает текст из XML файла."""
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()
        full_text = []
        for element in root.iter():
            if element.text:
                full_text.append(element.text)
        return '\n'.join(full_text)
    except Exception as e:
        print(f"Ошибка при чтении XML файла: {e}")
        return None


def load_text_from_txt(file_path, encoding='windows-1251'):
    """Загружает текст из TXT файла."""
    try:
        with open(file_path, "r", encoding=encoding) as f:
            return f.read()
    except Exception as e:
        print(f"Ошибка при чтении TXT файла: {e}")
        return None

def load_document(file_path, encoding='windows-1251'): # default encoding
    """Загружает текст из файла, автоматически определяя формат."""
    file_extension = os.path.splitext(file_path)[1].lower()

    if file_extension == ".docx":
        return load_text_from_doc(file_path)
    elif file_extension == ".pdf":
        return load_text_from_pdf(file_path)
    elif file_extension == ".xml":
        return load_text_from_xml(file_path)
    elif file_extension == ".txt":
        return load_text_from_txt(file_path, encoding)
    else:
        print("Неподдерживаемый формат файла.")
        return None


In [ ]:
import re
import os

# 3. Функция разделения документа на чанки
def split_document(text, document_title):

    if not text:
        print("Текст для разделения отсутствует.")
        return []

    # Разделители
    chapter_separator = r"^\s*Глава\s+\d+\..*"
    article_separator = r"^\s*Статья\s+\d+\..*"

    # Результат - список чанков с метаданными
    chunks = []
    global_metadata = {"document_title": document_title}

    # Разделяем на главы и статьи
    parts = re.split(f"({chapter_separator}|{article_separator})", text, flags=re.MULTILINE)

    current_chapter = None
    current_chapter_description = ""
    current_article = None

    for i in range(1, len(parts), 2):  # Iterate through separators and content
        separator = parts[i]
        content = parts[i + 1].strip() if i + 1 < len(parts) else ""

        if re.match(chapter_separator, separator):
            current_chapter = separator.strip()
            current_chapter_description = content  # Chapter description is immediately after the chapter title
        elif re.match(article_separator, separator):
            current_article = separator.strip()
            chunk = {
                "page_content": content,
                "metadata": {
                    "global": global_metadata,
                    "chapter": current_chapter,
                    "chapter_description": current_chapter_description,
                    "article": current_article
                }
            }
            chunks.append(chunk)
            current_chapter_description = "" # reset
        else:  # Content without a separator, usually part of a previous article
             if chunks: # if we already have an article
                chunks[-1]["page_content"] += "\n" + content # append to last article
             elif current_chapter_description: # if no article, but have chapter desc
                 current_chapter_description += "\n" + content #append to chapter desc
             else:
                 print(f"Warning: orphaned text found: {content[:50]}...") # orphan.

    return chunks


def save_chunks_to_txt(chunks, file_path):
    """Сохраняет список чанков в файл в формате TXT."""
    try:
        with open(file_path, "w", encoding="utf-8") as f:
            for i, chunk in enumerate(chunks):
                f.write(f"Chunk {i+1}:\n")
                f.write(f"  page_content: {chunk['page_content']}\n")
                f.write("  metadata:\n")
                for key, value in chunk['metadata'].items():
                    if isinstance(value, dict):
                        f.write(f"    {key}:\n")
                        for sub_key, sub_value in value.items():
                            f.write(f"      {sub_key}: {sub_value}\n")
                    else:
                        f.write(f"    {key}: {value}\n")
                f.write("-" * 40 + "\n")
        print(f"Чанки сохранены в файл: {file_path}")
    except Exception as e:
        print(f"Ошибка при сохранении чанков в файл: {e}")

text = load_document(FILE_PATH, encoding='windows-1251')
document_title = os.path.basename(FILE_PATH) # or "Your Document Title"
chunks = split_document(text, document_title)
save_chunks_to_txt(chunks, "output.txt")

# 4. Функция для сохранения чанков в файл

def save_chunks_to_file(chunks, file_path):
    """Сохраняет список чанков в файл в формате JSON."""
    try:
        with open(file_path, "w", encoding="utf-8") as f:
            json.dump(chunks, f, indent=4, ensure_ascii=False)  # Сохраняем в формате JSON
        print(f"Чанки сохранены в файл: {file_path}")
    except Exception as e:
        print(f"Ошибка при сохранении чанков в файл: {e}")

# 5. Основной код выполнения
# 1. Загружаем текст из документа
text = load_document(FILE_PATH, encoding='windows-1251')

if text:
    # 2. Разделяем текст на чанки
    chunks = split_document(text, document_title)

    if chunks:
        print(f"Найдено {len(chunks)} чанков.")

        # 3. Создаем папку для сохранения результатов
        file_name = os.path.splitext(os.path.basename(FILE_PATH))[0]
        output_folder_name = f"#{file_name}"
        output_folder_path = os.path.join(OUTPUT_BASE_PATH, output_folder_name)

        if not os.path.exists(output_folder_path):
            os.makedirs(output_folder_path)
            print(f"Создана папка: {output_folder_path}")

        # 4. Сохраняем чанки в файл
        output_file_path = os.path.join(output_folder_path, "chunks.json")
        save_chunks_to_file(chunks, output_file_path)

        # 5. Выводим пример чанка в удобочитаемом виде
        print("\nПример чанка:")
        example_chunk = chunks[10]

        print("page_content:")
        print(example_chunk["page_content"])

        print("\nmetadata:")
        for key, value in example_chunk["metadata"].items():
            print(f"  {key}: {value}")

    else:
        print("Чанки не найдены.")
else:
    print("Не удалось загрузить текст из документа.")

Чанки сохранены в файл: output.txt
Найдено 59 чанков.
Создана папка: /content/drive/MyDrive/СТАЖИРОВКА_ЖКХ/Отчёты/#ФЗ от 27.07.2010 № 190-ФЗ-О теплоснабжении
Чанки сохранены в файл: /content/drive/MyDrive/СТАЖИРОВКА_ЖКХ/Отчёты/#ФЗ от 27.07.2010 № 190-ФЗ-О теплоснабжении/chunks.json

Пример чанка:
page_content:
1. Регулированию подлежат, за исключением случаев, указанных в частях 2.1 - 2.3 настоящей статьи, с учетом статьи 23.4 настоящего Федерального закона следующие виды цен (тарифов) в сфере теплоснабжения:
1) - 2) утратили силу. - Федеральный закон от 26.02.2024 N 34-ФЗ;
3) тарифы на тепловую энергию (мощность), производимую в режиме комбинированной выработки электрической и тепловой энергии источниками тепловой энергии с установленной генерирующей мощностью производства электрической энергии 25 мегаватт и более;
4) тарифы на тепловую энергию (мощность), поставляемую теплоснабжающими организациями потребителям, а также тарифы на тепловую энергию (мощность), поставляемую теплоснабж

#@title 3. Embeddings

* Блок для загрузки чанков и создания FAISS индекса

output_dir   -->   Путь к папке с  чанками формат  JSON  


FILE_PATH   -->    Путь к папке с  Эмбеддингами формат   faiss  





In [ ]:
import os
import json
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# --- Блок для загрузки чанков и создания FAISS индекса ---

# Путь к папке с  Эмбеддингами формат   faiss
output_dir = "/content/drive/MyDrive/СТАЖИРОВКА_ЖКХ/Отчёты/Embeddings_FZ"

# Путь к папке, где находятся чанки (из предыдущего шага)
FILE_PATH = "/content/drive/MyDrive/СТАЖИРОВКА_ЖКХ/Отчёты/#ФЗ от 27_07_2006 N 152-ФЗ - О персональных данных" # Замените

file_name = os.path.splitext(os.path.basename(FILE_PATH))[0]
input_filename = "chunks.json"
input_filepath = os.path.join(FILE_PATH, input_filename)  # Используем FILE_PATH как базовую папку


try:
    with open(input_filepath, "r", encoding="utf-8") as f:
        chunks = json.load(f)  # Загружаем чанки из JSON
    print(f"Чанки загружены из: {input_filepath}")

    # Инициализируем модель эмбеддингов
    embeddings_model = OpenAIEmbeddings()

    # Создаем список текстов для эмбеддингов
    texts = [chunk["page_content"] for chunk in chunks]

    # Получаем эмбеддинги
    embeddings = embeddings_model.embed_documents(texts)

    # Создаем FAISS индекс
    db = FAISS.from_texts(texts, embeddings_model)

    print(f"Размер эмбеддинга: {len(embeddings[0])}")

    # --- Блок для сохранения результатов на OpenAI Drive ---
    #output_dir = "/content/drive/MyDrive/СТАЖИРОВКА_ЖКХ/Отчёты"
    os.makedirs(output_dir, exist_ok=True)  # Создаем директорию, если она не существует

    output_filename = f"Em{file_name}_faiss" # Используем имя файла из FILE_PATH
    output_filepath = os.path.join(output_dir, output_filename)

    db.save_local(output_filepath)  # Сохраняем FAISS индекс с помощью save_local

    print(f"Индекс FAISS сохранен в: {output_filepath}")

except Exception as e:
    print(f"Ошибка: {e}")
    db = None

Чанки загружены из: /content/drive/MyDrive/СТАЖИРОВКА_ЖКХ/Отчёты/#ФЗ от 27_07_2006 N 152-ФЗ - О персональных данных/chunks.json
Размер эмбеддинга: 1536
Индекс FAISS сохранен в: /content/drive/MyDrive/СТАЖИРОВКА_ЖКХ/Отчёты/Embeddings_FZ/Em#ФЗ от 27_07_2006 N 152-ФЗ - О персональных данных_faiss


#@title 4 Запуск модели



In [ ]:
from openai import OpenAI
import textwrap
import re

# Функция для форматирования текста по абзацам
def format_text(text, width=80):
    # Разделяем текст на абзацы
    paragraphs = text.split('\n')
    # Форматируем каждый абзац отдельно
    formatted_paragraphs = []
    for paragraph in paragraphs:
        # Используем textwrap.fill для форматирования абзаца, чтобы длина строки не превышала width
        formatted_paragraph = textwrap.fill(paragraph, width)
        formatted_paragraphs.append(formatted_paragraph)
    # Объединяем абзацы с символом новой строки
    return '\n'.join(formatted_paragraphs)

def answer_index(system, topic, search_index, document_title, verbose=True): # Добавим document_title
    """
    Отвечает на вопрос, используя предоставленную базу знаний и метаданные статей.

    Args:
        system: Системные инструкции для модели.
        topic: Вопрос пользователя.
        search_index: Индекс FAISS для поиска релевантных отрывков.
        document_title: Название исходного документа.
        verbose: Флаг для отображения отладочной информации.

    Returns:
        Ответ на вопрос с указанием источника (название документа, статья, и т.д.).
    """

    # Поиск релевантных отрезков из базы знаний
    docs = search_index.similarity_search(topic, k=4)

    if verbose:
        print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join(
        [f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))

    if verbose:
        print('message_content :\n ======================================== \n', message_content)

    client = OpenAI()
    messages = [
        {"role": "system", "content": system},
        {'role': 'assistant', 'content': assistant},
        {"role": "user",
         "content": f"Используя предоставленный документ, ответь на вопрос клиента. Не упоминай явно документ с информацией для ответа клиенту в ответе.\
         Документ с информацией для ответа клиенту: {message_content} \
         \n\nВопрос клиента: \n{topic}"}
    ]

    if verbose:
        print('\n ===========================================: ')

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.5
    )

    answer = completion.choices[0].message.content

    # Добавляем информацию об источнике
    source_info = f"\n\nОтвет основан на документе: {document_title}, "

    # Попытаемся извлечь название главы и статьи из метаданных первого документа
    if docs and docs[0].metadata:
        chapter = docs[0].metadata.get("chapter", "не указана")
        #chapter_description = docs[0].metadata.get("chapter_description", "не указано")
        source_info += f"Глава: {chapter}. "
        #source_info += f"Описание Главы: {chapter_description}. "

        # Попробуем найти номер статьи в page_content (если это статья)
        article_match = re.search(r"Статья\s+(\d+)", docs[0].page_content)
        if article_match:
            article_number = article_match.group(1)
            source_info += f"Статья: {article_number}."
    else:
        source_info += "Источник в документе не определен."

    return format_text(answer + source_info)  # Возвращает отформатированный ответ с источником

In [ ]:
system ='''Ты консультант контактного центра жилого комплекса в сфере ЖКХ. Задача проводить консультации и предлагать решения по вопросам клиента на
основе нормативно-правовой базы Гражданского кодекса Российской Федерации а тка же внутренних регламентов жилого комплекса.
Твоя задача отвечать на вопросы приоритетно из предоставленной базы документов в контексте соответствующих статей обязательно указывая
название документа , статью, часть, раздел статьи или нормативный акт.
Строго соблюдай условие: Если вопрос не относится к теме не выводи ни какие документы, дружелюбно напоминай что необходимо вернуться в русло разговора темы ЖКХ в отношении жилого комплекса.
Не придумывай ничего от себя, отвечай максимально точно по документу. Не упоминай Документ с информацией для ответа клиенту.
Клиент ничего не должен знать про Документ с информацией для ответа клиенту.'''

assistant = '''Не пересказывай полностью раздел текста, Отвечай максимально точно по документу но коротко по сути вопроса.
Выделяй основные тезисы с обязательным указнием названия документа, статью, часть, раздел статьи или нормативный акт.
Задавай уточняющие вопросы для конкретики.'''

# 5. Тестируем: вопрос - ответ

In [ ]:
# Получите название документа (например, из FILE_PATH)
document_title = os.path.basename(FILE_PATH)

# вопрос пользователя
topic = "Расскажио о требованиях защиты персональных данных"
ans = answer_index(system, topic, db, document_title)  # Передаем document_title
print(format_text(ans))


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================1. Оператор при обработке персональных данных обязан принимать необходимые правовые, организационные и технические меры или обеспечивать их принятие для защиты персональных данных от неправомерного или случайного доступа к ним, уничтожения, изменения, блокирования, копирования, предоставления, распространения персональных данных, а также от иных неправомерных действий в отношении персональных данных.
2. Обеспечение безопасности персональных данных достигается, в частности:
1) определением угроз безопасности персональных данных при их обработке в информационных системах персональных данных;
2) применением организационных и технических мер по обеспечению безопасности персональных данных при их обработке в информационных системах персональных данных, необходимых для выполнения требований к защите персональных данных, исполнение которых обеспечивает установленные Правительством Рос

In [ ]:
# Получите название документа (например, из FILE_PATH)
document_title = os.path.basename(FILE_PATH)

# вопрос пользователя
topic = " действия в отношении лиц виновных в нарушении требований настоящего Федерального закона"
ans = answer_index(system, topic, db, document_title)  # Передаем document_title
print(format_text(ans))


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================1. Лица, виновные в нарушении требований настоящего Федерального закона, несут предусмотренную законодательством Российской Федерации ответственность.
2. Моральный вред, причиненный субъекту персональных данных вследствие нарушения его прав, нарушения правил обработки персональных данных, установленных настоящим Федеральным законом, а также требований к защите персональных данных, установленных в соответствии с настоящим Федеральным законом, подлежит возмещению в соответствии с законодательством Российской Федерации. Возмещение морального вреда осуществляется независимо от возмещения имущественного вреда и понесенных субъектом персональных данных убытков.  
Отрывок документа №2
=====================1. Настоящим Федеральным законом регулируются отношения, связанные с обработкой персональных данных, осуществляемой федеральными органами государственной власти, органами государстве

In [ ]:
# Получите название документа (например, из FILE_PATH)
document_title = os.path.basename(FILE_PATH)

# вопрос пользователя
topic = "я не хочу соблюдать требований настоящего Федерального закона"
ans = answer_index(system, topic, db, document_title)  # Передаем document_title
print(format_text(ans))


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================1. Законодательство Российской Федерации в области персональных данных основывается на Конституции Российской Федерации и международных договорах Российской Федерации и состоит из настоящего Федерального закона и других определяющих случаи и особенности обработки персональных данных федеральных законов.
2. На основании и во исполнение федеральных законов государственные органы, Банк России, органы местного самоуправления в пределах своих полномочий могут принимать нормативные правовые акты, нормативные акты, правовые акты (далее - нормативные правовые акты) по отдельным вопросам, касающимся обработки персональных данных. Такие акты не могут содержать положения, ограничивающие права субъектов персональных данных, устанавливающие не предусмотренные федеральными законами ограничения деятельности операторов или возлагающие на операторов не предусмотренные федеральными законами обяз

In [ ]:
# Получите название документа (например, из FILE_PATH)
document_title = os.path.basename(FILE_PATH)

# вопрос пользователя
topic = "Мне не нравится этот закон"
ans = answer_index(system, topic, db, document_title)  # Передаем document_title
print(format_text(ans))


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================Целью настоящего Федерального закона является обеспечение защиты прав и свобод человека и гражданина при обработке его персональных данных, в том числе защиты прав на неприкосновенность частной жизни, личную и семейную тайну.  
Отрывок документа №2
=====================1. Законодательство Российской Федерации в области персональных данных основывается на Конституции Российской Федерации и международных договорах Российской Федерации и состоит из настоящего Федерального закона и других определяющих случаи и особенности обработки персональных данных федеральных законов.
2. На основании и во исполнение федеральных законов государственные органы, Банк России, органы местного самоуправления в пределах своих полномочий могут принимать нормативные правовые акты, нормативные акты, правовые акты (далее - нормативные правовые акты) по отдельным вопросам, касающимся обработки персональных

In [ ]:
# Получите название документа (например, из FILE_PATH)
document_title = os.path.basename(FILE_PATH)

# вопрос пользователя
topic = "Пошёл ты"
ans = answer_index(system, topic, db, document_title)  # Передаем document_title
print(format_text(ans))


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================Операторы и иные лица, получившие доступ к персональным данным, обязаны не раскрывать третьим лицам и не распространять персональные данные без согласия субъекта персональных данных, если иное не предусмотрено федеральным законом.  
Отрывок документа №2
=====================1. Обработка персональных данных в целях продвижения товаров, работ, услуг на рынке путем осуществления прямых контактов с потенциальным потребителем с помощью средств связи, а также в целях политической агитации допускается только при условии предварительного согласия субъекта персональных данных. Указанная обработка персональных данных признается осуществляемой без предварительного согласия субъекта персональных данных, если оператор не докажет, что такое согласие было получено.
2. Оператор обязан немедленно прекратить по требованию субъекта персональных данных обработку его персональных данных, указанную 